In [2]:
import numpy as np
import scipy.sparse.linalg as splin
from scipy.sparse import diags, csc_matrix
import scipy.linalg as linalg
import matplotlib.pyplot as plt

# Closed form expression of eigenvalues for 5-point Laplacian

$$\begin{align*}
    \nabla^2u_{lj} &= \frac{u_{l+1,j} + u_{l-1,j} + u_{l,j+1} + u_{l,j-1} - 4u_{lj}}{h^2}
    \\ \\ 
    \text{Assume:} \qquad & 
    \\
        u_{ij} &= Ae^{i(k_xl +k_y j)}
    \\ \\
    \text{Insert in Laplacian} \qquad & 
    \\
    \lambda u_{lj} &= \frac{u_{l+1,j} + u_{l-1,j} + u_{l,j+1} + u_{l,j-1} - 4u_{lj}}{h^2}
    \\
    \lambda Ae^{i(k_xl + k_y j)} &= A\frac{e^{i(k_x(l+1) + k_yj)} + e^{i(k_x(l-1) + k_yj)} + e^{i(k_xl + k_y(j+1))} + e^{i(k_xl + k_y(j-1))} - 4 e^{i(k_xl+ k_yj)}}{h^2}
    \\
            &= A\frac{
                u_{lj}e^{ik_x} + u_{lj}e^{- ik_x} + u_{lj}e^{ ik_y} + u_{lj}e^{- ik_y} - 4 u_{lj}
            }{h^2}
    \\ \\
    \text{Solve for $\lambda$:} \qquad & \\
    \lambda &= \frac{
        e^{ik_x} + e^{-ik_x} + e^{ik_y} + e^{-ik_y} - 4
    }{h^2}
    \\
            &= \frac{2\cos{k_x} + 2\cos{k_y} - 4}{h^2}
\end{align*}$$

For grid-size $N\times N$ (including boundary) and Dirichlet BC the allowed wave numbers are

$$\begin{align*}
    k_x = \frac{\pi m}{N+1},\quad k_y = \frac{\pi n}{N+1}, \qquad m,n = 1,2,\ldots, N-1
\end{align*}$$

Resulting in the eigenvalues
$$
    \lambda_{mn} = \frac{2\cos{\frac{\pi m}{N+1}} + 2 \cos{\frac{\pi n}{N+1}} - 4}{h^2}
$$

In [181]:
def Amult(U, m):
    h = 1/(m+1)
    index = lambda x, y: x*m + y # row-wise indexing
    AU = np.zeros((m**2, ))
    for i in range(m): # i, j in [0, m-1] due to zero-indexing
        for j in range(m):
            AU[index(i,j)] -= 4*U[index(i, j)]
            
            # if-statements ensuring add element if within interior
            if i < m-1:
                AU[index(i,j)] +=   U[index(i+1, j)] 
            if i > 0:
                AU[index(i,j)] +=   U[index(i-1, j)] 
            if j < m-1:
                AU[index(i,j)] +=   U[index(i, j+1)] 
            if j > 0:
                AU[index(i,j)] +=   U[index(i, j-1)] 
    return AU / h**2

# number of interior points
m = 5

# our 2d grid intial guess
U2D = np.zeros((m,m))
U2D[:, 4] = 1
print(U2D)

# 1D representation
U = U2D.flatten()
print(U)
Amult(U, m).reshape(m, m)

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1.]


array([[   0.,    0.,    0.,   36., -108.],
       [   0.,    0.,    0.,   36.,  -72.],
       [   0.,    0.,    0.,   36.,  -72.],
       [   0.,    0.,    0.,   36.,  -72.],
       [   0.,    0.,    0.,   36., -108.]])